<a href="https://colab.research.google.com/github/gprasad125/lign167_finalproject/blob/main/Copy_of_Project_Tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Multiclass Text Classification to Analyze Famous Quotes 

#### Gokul Prasad & Hoang Nguyen 
#### LIGN 167, Winter 2022

In this project, we'll aim to classify a variety of quotes with tags that refer to certain themes or elements specific to that particular quote. 

For example, Albert Einstein's quote “Life is like riding a bicycle. To keep your balance, you must keep moving.” would have tags like "life" or "simile" because it contains thematic elements about life, and contains a simile. 

In [25]:
# Scraping
import requests
from bs4 import BeautifulSoup
import time 

# Data manipulation / cleaning / visualization
import pandas as pd
import numpy as np
import gensim as gm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

import re 
import matplotlib.pyplot as plt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Sklearn modeling
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

# keras / tensorflow modeling
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, TextVectorization, StringLookup
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.metrics import Precision, Recall

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.data import AUTOTUNE
from tensorflow.data import Dataset
from tensorflow.ragged import constant
from tensorflow.keras import layers

# Transformers for model 2.2
!pip install transformers
import transformers
from transformers import AutoTokenizer,TFBertModel

from tensorflow.keras.optimizers import SGD

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Scraping and Cleaning the Data 

We'll be sourcing our data from https://www.goodreads.com/quotes. This is a website containing 100 pages worth of quotes, each of them classified with a few tags. 

Firstly, we'll loop through the pages, and scrape the website HTML data with BeautifulSoup. Then, we'll use lambda functions to pull author data, quote data, and tag data. We'll put each of these into lists, and then create a pandas DataFrame to hold all our data. 

In [2]:
goodreads_quotes = []
goodreads_tags = []

for i in range(1, 101):

  url = 'https://www.goodreads.com/quotes?page={}'.format(i)

  time.sleep(5)
  scrape = requests.get(url)
  parsed = BeautifulSoup(scrape.content, 'html.parser')

  elements_quotes = parsed.find_all('div', class_ = "quoteText")
  
  quotes = [x.text.strip() for x in elements_quotes]
  tags = parsed.find_all(class_ = 'quoteFooter')

  goodreads_quotes += quotes
  goodreads_tags += tags

In [3]:
data = {'quote':goodreads_quotes, 'tags':goodreads_tags}
goodreads = pd.DataFrame(data)
goodreads.head()

,quote,tags
0,“Be yourself; everyone else is already taken.”...,"[\n, [\n tags:\n , [attributed-no-so..."
1,"“I'm selfish, impatient and a little insecure....","[\n, [\n tags:\n , [attributed-no-so..."
2,“Two things are infinite: the universe and hum...,"[\n, [\n tags:\n , [attributed-no-so..."
3,"“So many books, so little time.”\n ―\n \n ...","[\n, [\n tags:\n , [books], ,\n ..."
4,“A room without books is like a body without a...,"[\n, [\n tags:\n , [attributed-no-so..."


As we can see, our dataset contains some pretty messy strings in both columns. We'll need to process the data to make sure it's usable for our modeling later on. 

For quotes, we'll first make all characters lowercase, and then use regex functionality to substitute any non alphanumeric / whitespace character with a blank string. 

For example, if we input a quote like "I love. LIGN \n167!!?   \n \nOscar Wilde " we would receive an output of "i love lign 167". We'll apply this to our Author and Quote columns to clean them up and make them much more simplified strings. 

In [4]:
def quotes_cleaning(text):
    
    text = text.lower()
    
    text = re.sub('[^A-Za-z0-9\s]', '', text)
    
    return text

def tags_cleaning(text):
    
    text = re.sub('[\[ \]]', ' ', str(text))
    text = re.sub('[^\w]', ' ', text)
    text = re.sub('[\s]', ' ', text)
    text = re.sub('[0-9]', ' ', text)
    
    text = ' '.join(text.split())
    
    return text.split(' ')

def remove_author(quote):

  if quote[0] == '“':

    end_of_quote = quote.index('”')
    quote = quote[1:end_of_quote]

  return quote

def bs_to_list(tags):

  if type(tags) != list:

    tags = tags.find_all('a')
  
    tag_strs = []
    for tag in tags[:-1]:

      tag = str(tag)
      start_idx = tag.index('">')
      end_idx = tag.index('</')
      tag = tag[start_idx + 2:end_idx]
      tag_strs.append(tag)

    tags = tag_strs

  return tags

For the tags, we have to a slightly more complicated function since the data is tucked into lists. Firstly, we'll make it a string, and use regex to remove the surrounding brackets, remove non-word characters, and replace all multi-whitespaces with a single space. We'll then render the string as a list again, and return the list. 

For example, if we input a list like [deep?, wonderous.., love-happy], we would get an output of [deep, wonderous, love, happy]

In [5]:
goodreads['quote'] = goodreads['quote'].apply(remove_author)
goodreads['quote'] = goodreads['quote'].apply(quotes_cleaning)
goodreads['tags'] = goodreads['tags'].apply(bs_to_list)
goodreads['tags'] = goodreads['tags'].apply(tags_cleaning)

isEmpty = goodreads['tags'].apply(lambda x: '' in x)
goodreads['isEmpty'] = isEmpty

Now, having cleaned the dataset more fully, we can see the impact on our data. We've also added an isEmpty column, which marks whether or not the quote has no tags. Since the tags are in a list, empty lists will exist as "[ ]" and not as a NaN value.  

In [6]:
goodreads.head(4)

,quote,tags,isEmpty
0,be yourself everyone else is already taken,"[attributed, no, source, be, yourself, gilbert...",False
1,im selfish impatient and a little insecure i m...,"[attributed, no, source, best, life, love, mis...",False
2,two things are infinite the universe and human...,"[attributed, no, source, human, nature, humor,...",False
3,so many books so little time,"[books, humor]",False


# Reshaping Data for Modeling 

Now, while the data is cleaned, we can't really model accurately when our tags are all in a list. Inputting them into our sklearn Pipelines later would not work as we would want, so we have to find a way to reshape the dataframe. Firstly, we'll need to collect the minimum and maximum amount of tags, which we do as follows. 

In [ ]:
max_tags = goodreads['tags'].apply(lambda x: len(x)).max()
max_tags

So we see that the maximum amount of tags a quote could have would be 48 tags. So, let's generate a function that will make each list of tags equivalent by adding the necessary number of None values to make it to a list of length 48. 

For example, an input of [life, duck, nature] would yield [life, duck, nature, None, None, None, None, None, None, None, None, ... None, None, None]. 

In [ ]:
def pad(tags):

  needed = 48 - len(tags)
  tags = tags + ([None] * needed)

  return tags

Now we can apply that function to our Tags column, and use pandas get_dummies() functionality to reshape our dataframe to where each tag is a column, and the column contains 1s or 0s, reflecting whether or not a particular tag is in the quote belonging to that row. 

Unfortunately, pd.get_dummies() will create some duplicates so we'll groupby and sum to combine the duplicate tag columns. 

We then combine this dataframe with our original dataframe, and drop our tags columns. We can see the finished result

In [ ]:
gr_tags = pd.DataFrame(goodreads['tags'].apply(pad).tolist())
gr_tags_oh = pd.get_dummies(gr_tags, prefix = 'tags')
gr_tags_oh = gr_tags_oh.groupby(gr_tags_oh.columns, axis = 1).sum()
reshaped_gr = pd.concat([goodreads, gr_tags_oh], axis = 1).drop(columns = ['isEmpty', 'tags'])
reshaped_gr.head()

We can see the distribution of tags as below.

In [ ]:
cnts = reshaped_gr.iloc[:, 1:].sum(axis = 1)
cnts.hist()

# Modeling

Now, we can begin our modeling. 

Firstly, we'll get a list of all of our tags. We'll do this by taking all columns besides "Quote"

Next, we'll use sklearn's train_test_split() function to split our dataset into a training and testing set. We'll split so that our test set is 33% of our dataset size. As we have 100 rows into our data, then we'll have a training set of 67 rows and testing size of 33 rows. 

In [ ]:
gr_tags = reshaped_gr.columns[1:]

train, test = train_test_split(reshaped_gr, test_size = 0.25, random_state = 42)

x_tr = train.quote
x_te = test.quote

print(x_tr.shape)
print(x_te.shape)

### Model 1: Decision Tree Classifier

Our first model will be using scikit-learn Pipelines. 

Inside our pipeline, we'll firstly vectorize the input data by converting the quote to their TFIDF formation. This will convert our string Quotes to becoming numerical values for input. Then, we have to consider how we will be handling multiple classes. We'll try with a OneVsRest classifier, because this will allow us to pass in each tag and use an single-class estimator on each tag's train and test data. 

However, we need to wrap the OneVsRest classifier around an estimator that makes sense for what we are trying to achieve here. We'll use a Decision tree classifier, because the sklearn functionality is pretty simplistic, doesnt require much shaping of the data, and should hopefully set a good basis for our first try. 

In [ ]:
dt_classifier = Pipeline([('tfidf', TfidfVectorizer()), ('clf', OneVsRestClassifier(DecisionTreeClassifier()))])

Now, we'll loop through each of the tags in our dataset, train our model on that particular tag, and then append it to a dictionary containg each tag and that tag's associated evaluation score. 

For our evaluating metric, we'll choose to use f1 scores over accuracy, because if we look at our data, we have an imbalance of tags. Some quotes have several tags, while others only have one or two. As such, using accuracy would likely not work well for this scenario. 

However, we have multiple classes, so it would not make much sense to get a bunch of f1 scores since each tag would give different results. We can instead collect each tag's precision and recall from when the model's predictions are compared to the actual test data. 

In [ ]:
prec_recs = {}
for tag in gr_tags:
    
    dt_classifier.fit(x_tr, train[tag])
    prediction = dt_classifier.predict(x_te)
    
    precision_recall = precision_recall_fscore_support(test[tag], prediction, average = 'macro')
    prec_recs[tag] = precision_recall

So now we can calculate the average precision and recall for our tags by looping through our dictionary, summing up the total of both metrics, and dividing by the number of tags.

In [ ]:
sum_precision = 0
sum_recall = 0

for key in prec_recs.keys():
    
    sum_precision += prec_recs[key][0]
    sum_recall += prec_recs[key][1]
    
mean_precision = sum_precision / len(prec_recs.keys())
mean_recall = sum_recall / len(prec_recs.keys())

Now we apply the formula of finding an f1 score which is (2 * p * r) / (p + r)

In [ ]:
average_f1 = (2 * mean_precision * mean_recall) / (mean_precision + mean_recall)
average_f1

So we have an f1 score of about 0.708. F1 scores range from 0 to 1, and the closer they are to 1, the better the model, so we have set up a good baseline for ourselves. But we want to improve on this and make our model better classify our quotes. 

#### Optimizing Model 1  

Now that we have our baseline model, how can we optimize it? 

There are many concepts we can implement into our Pipeline, both from a text classification standpoint, as well as a sklearn standpoint. 

The first method we'll implement is getting rid of stop-words. These are words that appear extremely frequently in human language, and give very little value to our model. Removing them can allow our model to focus more strongly on the more important data. 

In [ ]:
stop_words = set(stopwords.words('english'))

Now that we have defined the words to remove, we can try and optimize our other parameters with GridSearchCV. First, we'll need to select what parameters we can optimize. 

In [ ]:
parameters = {
    'clf':(DecisionTreeClassifier(),),
    'clf__max_depth': [2, 3, 4, 5, 7, 10, 13, 15, 18, None],
    'clf__min_samples_split': [2, 3, 5, 7, 10, 15, 20],
    'clf__min_samples_leaf': [2, 3, 5, 7, 10, 15, 20]
}

Now we have created the parameters, we can place that into a GridSearchCV and train it on our data. 
Let's print out the best parameters we get. 

In [ ]:
grids = GridSearchCV(dt_classifier, param_grid = parameters, cv = 3, return_train_score = True)
for tag in gr_tags:
    grids.fit(x_tr, train[tag])

grids.best_params_

Let's now re-run our training, testing, and calculating of precision and recall to calculate a new and hopefully improved average f1 score. 

In [ ]:
dt_classifier = Pipeline([('tdidf', TfidfVectorizer(stop_words = stop_words)), ('dtc', DecisionTreeClassifier(max_depth = 2, min_samples_leaf = 2))])

prec_recs = {}
for tag in gr_tags:
    
    dt_classifier.fit(x_tr, train[tag])
    prediction = dt_classifier.predict(x_te)
    
    precision_recall = precision_recall_fscore_support(test[tag], prediction, average = 'macro')
    prec_recs[tag] = precision_recall

sum_precision = 0
sum_recall = 0

for key in prec_recs.keys():
    
    sum_precision += prec_recs[key][0]
    sum_recall += prec_recs[key][1]
    
mean_precision = sum_precision / len(prec_recs.keys())
mean_recall = sum_recall / len(prec_recs.keys())

average_f1 = (2 * mean_precision * mean_recall) / (mean_precision + mean_recall)
average_f1

So we see a decent improvement from 0.71 --> 0.78, achieved with GridSearchCV and stop_word inclusion to optimize our model. However, we'll take a look at other models / optimizations to see if we can get a heightened score. 

### Model 2: 

The next model we'll try is a Keras Sequential() model with a couple layers. Firstly, we'll already go ahead and remove stopwords from both the quotes and tags, and append these to new columns. 

In [87]:
goodreads = goodreads[goodreads['isEmpty'] == False]
goodreads['stop_quote'] = goodreads['quote'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
goodreads['stop_tags'] = goodreads['tags'].apply(lambda x: [z for z in x if z not in stop_words])

goodreads.head()

,quote,tags,isEmpty,stop_quote,stop_tags
0,be yourself everyone else is already taken,"[attributed, no, source, be, yourself, gilbert...",False,everyone else already taken,"[attributed, source, gilbert, perreira, honest..."
1,im selfish impatient and a little insecure i m...,"[attributed, no, source, best, life, love, mis...",False,im selfish impatient little insecure make mist...,"[attributed, source, best, life, love, mistake..."
2,two things are infinite the universe and human...,"[attributed, no, source, human, nature, humor,...",False,two things infinite universe human stupidity i...,"[attributed, source, human, nature, humor, inf..."
3,so many books so little time,"[books, humor]",False,many books little time,"[books, humor]"
4,a room without books is like a body without a ...,"[attributed, no, source, books, simile, soul]",False,room without books like body without soul,"[attributed, source, books, simile, soul]"


We'll shuffle the data using pandas sample() functions, and take only our new stop-word removed quotes and tags columns. Then we'll split training and testing data on an 80/20 split, and get validation data as half the test data.

The shapes of the data are as follows:

In [102]:
goodreads_sample = goodreads.sample(frac = 1)
goodreads_sample = goodreads_sample[['stop_tags', 'stop_quote']]
train, test = train_test_split(goodreads_sample, test_size = 0.2, shuffle = True)
val = test.sample(frac=0.5)
test.drop(val.index, inplace=True)
train.shape, test.shape, val.shape

((2002, 2), (251, 2), (250, 2))

Next, we'll encode the list of strings in the "stop_tags" column to shift them from just being Strings to an integer output. We'll accomplish this via keras' constant() and StringLookup() functionality. Since we have multiple tags per quote, we need to use multi-hot encoding, since it allows for multiple markers to have a 1 (reflecting presence) indicating multiple labels. 

In [103]:
terms = constant(train["stop_tags"].values)
lookup = StringLookup(output_mode = "multi_hot")
lookup.adapt(terms)

We're going to need some information about our quote data, so we'll quickly split them into lists and use pandas describe() methods to generate the info about max lengths, avg lengths, etc.

In [104]:
train["stop_quote"].apply(lambda x: len(x.split(" "))).describe()

count    2002.000000
mean       14.582917
std        22.414865
min         1.000000
25%         6.000000
50%         9.000000
75%        15.000000
max       348.000000
Name: stop_quote, dtype: float64

Now, we're going to convert our training / testing / validation data from pandas DataFrames to keras Batch Datasets, because the latter will split up the datasets into chunks of a given size, which makes feeding it into our model much easier. 

In [111]:
def convert_to_tensors(df, batch_size = 6):

    labels = constant(df["stop_tags"].values)
    label_binarized = lookup(labels).numpy()

    tensor_df = Dataset.from_tensor_slices((df["stop_quote"].values, label_binarized))
    batch_df = tensor_df.batch(batch_size)
    return batch_df

train_dataset = convert_to_tensors(train)
validation_dataset = convert_to_tensors(val)
test_dataset = convert_to_tensors(test)

In [112]:
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 1819), dtype=tf.float32, name=None))>

Let's create a set of our unique words in our quotes column. We'll create a blank set and append the words to it. The size is as follows:

In [113]:
vocabulary = set()

total_words = train["stop_quote"].str.lower().str.split()
total_words.apply(vocabulary.update)

num_words = len(vocabulary)
print(num_words)

6791


Now, we have to represent our strings as vectors so that they can be fed into the model. Like our first model, we will use a TFIDF representation, conducted through Keras' TextVectorization function. We will have our maximum number of tokens set to as many unique words we have, and we will set ngrams = 2 because they will help the model understand context of related words. 


Then, we'll adapt it to our training dataset to generate the vocabulary from our quotes column. 

Now we can remake our datasets with our quotes converted to TFIDF-representation vectors. We do this by mapping the tensor dataframes's columns to a tfidf-representation quote and tags. 

In [114]:
tfidf_vect = TextVectorization(max_tokens = num_words, ngrams = 2, output_mode = "tf_idf")
tfidf_vect.adapt(train_dataset.map(lambda quote, tags: quote))

train_dataset = train_dataset.map(lambda quote, tags: (tfidf_vect(quote), tags))
validation_dataset = validation_dataset.map(lambda quote, tags: (tfidf_vect(quote), tags))
test_dataset = test_dataset.map(lambda quote, tags: (tfidf_vect(quote), tags))

Now we'll build the Keras Sequential() model with a few different layers:

- Firstly, we'll include a Dropout layer. This will help combat overfitting by randomly setting some input variables to 0 for ignoring. 

- Next, we'll add a generic Dense layer that will take in 1000 values, and a ReLU activation function. 

- We'll do this again in the next layer; however, this time we'll input half of our prior layer. 

- Finally, we'll get our output layer with a size of how many unique tags we have. We'll use a sigmoid activation function because we are dealing with a classification problem. 

Compilation:

For any particular given quote, they can have multiple tags. As such, we'll use a binary cross entropy loss function, since the model will attempt to predict whether or not a particular observation (Quote) belongs to any particular class of classes (Tags).

We will use a baseline Adam optimizer, which ..., and we will use metrics of precision and recall to calculate an f1 score, which we want to use given the imbalance of classes. 

In [115]:
keras_model = Sequential()
keras_model.add(Dropout(0.2))
keras_model.add(Dense(1000, activation = 'relu'))
keras_model.add(Dense(500, activation = 'relu'))
keras_model.add(Dense(lookup.vocabulary_size(), activation = 'sigmoid'))

loss = 'binary_crossentropy'
opt = Adam()

keras_model.compile(loss = loss, optimizer = opt, metrics = [Precision(), Recall()])
keras_model.build((None, num_words))
keras_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_11 (Dropout)        (None, 6791)              0         
                                                                 
 dense_33 (Dense)            (None, 1000)              6792000   
                                                                 
 dense_34 (Dense)            (None, 500)               500500    
                                                                 
 dense_35 (Dense)            (None, 1819)              911319    
                                                                 
Total params: 8,203,819
Trainable params: 8,203,819
Non-trainable params: 0
_________________________________________________________________


Now we can fit the Keras model onto our training dataset, which we will do so over 5 epochs. 

In [116]:
history = keras_model.fit(train_dataset, validation_data = validation_dataset, epochs = 5)

Epoch 1/5
334/334 [==============================] - 18s 51ms/step - loss: 0.0349 - precision_11: 0.0031 - recall_11: 0.0052 - val_loss: 0.0218 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 2/5
334/334 [==============================] - 17s 51ms/step - loss: 0.0158 - precision_11: 0.7419 - recall_11: 0.0032 - val_loss: 0.0189 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 3/5
334/334 [==============================] - 17s 50ms/step - loss: 0.0124 - precision_11: 0.8400 - recall_11: 0.0265 - val_loss: 0.0145 - val_precision_11: 0.1500 - val_recall_11: 0.0036
Epoch 4/5
334/334 [==============================] - 17s 50ms/step - loss: 0.0098 - precision_11: 0.8593 - recall_11: 0.0951 - val_loss: 0.0129 - val_precision_11: 0.5185 - val_recall_11: 0.0167
Epoch 5/5
334/334 [==============================] - 17s 50ms/step - loss: 0.0075 - precision_11: 0.8799 - recall_11: 0.2098 - val_loss: 0.0132 - val_precision_11: 0.3168 - val_recall_11: 0.0382


And we'll evaluate on the test dataset and see what we get for our precision and recall. 

In [117]:
info = keras_model.evaluate(test_dataset)

42/42 [==============================] - 1s 12ms/step - loss: 0.0134 - precision_11: 0.3368 - recall_11: 0.0387


And we calculate the F1 score. 

In [118]:
p = info[1]
r = info[2]

f1 = (2 * p * r) / (p + r)
f1

0.0694896881163711

Despite a middling precision, our recall is very low, and so our F1 score is as well, since the F1 is just the harmonic mean of our two given metrics. The fraction of correct predictions that the model guessed out of the total possible is too low for us to consider this a success. 

#### Random Single-Tag Classification

Given the unsuccessful nature of using a list of tags, let's try a completely different approach and see if we can make a better result, albeit from a different standpoint. 

Instead of trying to classify from a multilabel view, what if we just assign one tag to each quote? We saw earlier than many tags are repeated throughout the quotes, so random selection of one would simplify the problem. 

We'll apply this idea by using numpy's random.choice() method on each list of tags and putting that new tag in a column called 'rand_tag'. 

In [ ]:
subset = goodreads[['stop_quote', 'stop_tags']]
subset['rand_tag'] = subset['stop_tags'].apply(lambda x: np.random.choice(x))
subset.head()

,stop_quote,stop_tags,rand_tag
0,everyone else already taken,"[attributed, source, gilbert, perreira, honest...",inspirational
1,im selfish impatient little insecure make mist...,"[attributed, source, best, life, love, mistake...",control
2,two things infinite universe human stupidity i...,"[attributed, source, human, nature, humor, inf...",human
3,many books little time,"[books, humor]",humor
4,room without books like body without soul,"[attributed, source, books, simile, soul]",source


Next, we'll encode the tag by looping through the tags and placing them in a dictionary. If the tag exists in the dictionary, then nothing is done; however, if not, then it is added, and the number assigned is the new length of the dictionary. 

For example, a group of lists like:

[['life', 'love', 'data']

['love', 'joy', 'science']] 

would generate a dictionary of 

{'life': 1, 'love': 2, 'data': 3, 'joy': 4, 'science': 5}

In [ ]:
encoded = {}

for tag in subset.rand_tag:

  if tag not in encoded.keys():

    encoded[tag] = len(encoded)

subset['encoded_tag'] = subset.rand_tag.map(encoded)

Next, we'll shuffle the data again and reset the index. 

In [ ]:
subset = subset.sample(frac = 1).reset_index(drop = True)
subset.head()

,stop_quote,stop_tags,rand_tag,encoded_tag
0,dreams touchstones characters,"[characters, dreams, touchstones]",characters,816
1,past beats inside like second heart,"[memories, past]",past,699
2,dance youre broken open dance youve torn banda...,"[dance, inspirational]",dance,181
3,say weep tears evil,"[gandalf, grief]",grief,173
4,perhaps many causes worth dying certainly none...,"[martyrdom, misattributed, albert, camus, paci...",war,357


Then we'll onehot encode the tags via keras' to_categorical() function. This will take a Series of integers and convert it to arrays of 1 (present) and 0s (not present). 

In [ ]:
onehot_encoded = to_categorical(subset.encoded_tag)
onehot_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

We're going to use a AutoTokenizer to help tokenize our quotes later on, as well as a pretrained BERT model. 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


We'll split into training and testing data, with a split of 80/20 on our stop_quote and encoded_tag columns. 

We'll again find out information to inform our model via pandas describe() functionality. 

In [ ]:
data = subset[['stop_quote', 'encoded_tag']]
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
subset["stop_quote"].apply(lambda x: len(x.split(" "))).describe()

count    2475.000000
mean       14.407677
std        21.539474
min         1.000000
25%         6.000000
50%         9.000000
75%        15.000000
max       348.000000
Name: stop_quote, dtype: float64

Now we'll develop our x values from our training and testing datasets and tokenize them via the tokenizer initialization. 

We have several parameters to use:
- max_length is based off the information we saw earlier. This informs why we have truncation = True
- return_attention_mask = True. This informs the model not to pay attention to the special padding tokens when it reads the sentence. 

In [ ]:
x_train = tokenizer(text = train['stop_quote'].tolist(), max_length = 9, truncation = True, padding = True, return_tensors = 'tf', return_token_type_ids = False, return_attention_mask = True)
x_test = tokenizer(text = test['stop_quote'].tolist(), max_length = 9, truncation = True, padding = True, return_tensors = 'tf', return_token_type_ids = False, return_attention_mask = True)


Next, we can generate our y values from our training and testing datasets.

In [ ]:
y_train = train['encoded_tag']
y_test = test['encoded_tag']

Now, we begin building our model. 

Firstly, we need to build 2 input layers – one for our input tokens, and one for attention masks. The first is a list of tokens that we are feeding into our model, and the second is a list of indices that the model should ignore (i.e., special tokens)

Next, we'll input these into a BERT model for embedding. 

Now, we have to build the actual Keras model. 

1. Firstly, we'll use a GlobalMaxPool of 1 Dimension as the first layer. This will return the maximum value of the input ids (notwithstanding the indexes for the special ids). 

2. Next, we'll include a Dense layer of size 256, and we'll use a ReLU activation function. ReLU will take the input variable, and return the same variable if it is positive, and 0 otherwise. We choose to use ReLU here because the function is simple, and so the model will learn more quickly. Furthermore, it is generally applicable for many kinds of tasks.  

3. Next, we will have a Dropout layer. The layer randomly shifts some values to 0 at a rate of 0.1. This will be activated in the last line where we set the layer as trainable, so the values are preserved, and not dropped, when they are turned to 0. We use a Dropout layer to prevent overfitting, since the size of the dataset is relatively small for a whole Keras NN.

4. Next, we'll have another Dense layer, this time of size 64. We use another ReLU activation function. 

5. Finally, we'll use a single-value Dense layer to predict the y value with a sigmoid activation function. Sigmoid functions work well in cases of classification, which we are aiming to do here, so we want to use it in the final layer when the model is generating the final values of prediction. 



In [ ]:
max_len = 9
ids = Input(shape = (max_len,), dtype = tf.int32, name = "ids")
masks = Input(shape = (max_len,), dtype = tf.int32, name = "masks")

bert_embedded = bert(ids, attention_mask = masks)[0]
maxpool = tf.keras.layers.GlobalMaxPool1D()(bert_embedded)
dense_1 = Dense(256, activation = 'relu')(maxpool)
dropout = tf.keras.layers.Dropout(0.1)(dense_1)
dense_2 = Dense(64, activation = 'relu')(dropout)
prediction = Dense(1, activation = 'sigmoid')(dense_2)


model = tf.keras.Model(inputs=[ids, masks], outputs = prediction)
model.layers[2].trainable = True

Now let's print out the summary of the model

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ids (InputLayer)               [(None, 9)]          0           []                               
                                                                                                  
 masks (InputLayer)             [(None, 9)]          0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['ids[0][0]',                    
                                thPoolingAndCrossAt               'masks[0][0]']                  
                                tentions(last_hidde                                               
                                n_state=(None, 9, 7                                         

Now, we have to compile the model, with an optimizer and loss function. 

For our optimizer, we will use an Adam optimizer, which is an optimizer based off of stochastic gradient descent. We choose to use this optimizer because ...

Next, for our loss, we will use Categorical Crossentropy, because we have data where each value could have one out of any number of tags. Therefore, this loss function will best suit our model. From [Google](https://developers.google.com/machine-learning/glossary/#logits), logits are "the vector of raw (non-normalized) predictions that a classification model generates" which is our y-value so the from_logits parameter must be True.

Now, we'll run compile() with our generated optimizer and loss functions. We'll select Precision and Recall as our returned metrics so that we can calculate an F1 score. F1 scores work best for imbalanced classes which we would have here. 

In [ ]:
adam = Adam(learning_rate = 5e-05, epsilon = 1e-08, decay = 0.01, clipnorm=1.0)
loss = CategoricalCrossentropy(from_logits = True)

model.compile(optimizer = adam, loss = loss, metrics = [Precision(), Recall()])

Firstly, we have to make a dictionary of our 2 input values since we have to pass in the 'input_ids' and the 'attention_mask' data points. Next, we'll also preemptively set our validation data by pulling it from our test dataset. 

Then we'll fit the model to our x, y_train, val data over 5 epochs with a batch_size of 32. 

In [ ]:
x = {'ids': x_train['input_ids'], 'masks': x_train['attention_mask']}
val = ({'ids': x_test['input_ids'], 'masks': x_test['attention_mask']}, y_test)

train_history = model.fit(x = x, y = y_train, validation_data = val, epochs = 5)

Epoch 1/5
62/62 [==============================] - 204s 3s/step - loss: 0.0000e+00 - precision_1: 0.9433 - recall_1: 0.9888 - val_loss: 0.0000e+00 - val_precision_1: 0.9596 - val_recall_1: 1.0000
Epoch 2/5
62/62 [==============================] - 159s 3s/step - loss: 0.0000e+00 - precision_1: 0.9434 - recall_1: 1.0000 - val_loss: 0.0000e+00 - val_precision_1: 0.9596 - val_recall_1: 1.0000
Epoch 3/5
62/62 [==============================] - 162s 3s/step - loss: 0.0000e+00 - precision_1: 0.9434 - recall_1: 1.0000 - val_loss: 0.0000e+00 - val_precision_1: 0.9596 - val_recall_1: 1.0000
Epoch 4/5
62/62 [==============================] - 159s 3s/step - loss: 0.0000e+00 - precision_1: 0.9434 - recall_1: 1.0000 - val_loss: 0.0000e+00 - val_precision_1: 0.9596 - val_recall_1: 1.0000
Epoch 5/5
62/62 [==============================] - 162s 3s/step - loss: 0.0000e+00 - precision_1: 0.9434 - recall_1: 1.0000 - val_loss: 0.0000e+00 - val_precision_1: 0.9596 - val_recall_1: 1.0000


Now that the model is fitted, we'll run it on our testing data. 

In [ ]:
test_x = {'ids':x_test['input_ids'],'masks':x_test['attention_mask']}

test_info = model.evaluate(test_x, y_test)

16/16 [==============================] - 10s 649ms/step - loss: 0.0000e+00 - precision_1: 0.9596 - recall_1: 1.0000


Now, we can finally calculate our final F1 score, and we see the major improvement with this model that we achieved. 

In [ ]:
p = test_info[1]
r = test_info[2]

f1_score = (2 * p * r) / (p + r)
f1_score

0.9793814530198031

We see a much improved metric of success, proving that our model is a success. 